# First fermionic DMRG

Created 03/01/2025

Objectives:
* Construct the 1D fermionic spt states from the Wang & Gu paper, first pass. Work out the case with trivial projective representation first.
* Not sure of anyway to test the states...? Apart from running the SPT classification on them.

# Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [132]:
from itertools import product

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.site import ClockSite, FermionSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel

In [5]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [6]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [7]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [34]:
np.array([
    [
        mod_4_to_bit_addition(x, y) for x in range(4)
    ]
    for y in range(4)
])

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])

In [176]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [177]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [178]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    left_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_left, g_in), n1_func(g_left, g_out))
    ]

    out.append((left_fermion_op_string, 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    right_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_in, g_right), n1_func(g_out, g_right))
    ]

    out.append((right_fermion_op_string, 1, 1))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [179]:
class ClusterIsingFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                self.add_multi_coupling(-1/4, op_list)

In [180]:
M = ClusterIsingFermion({'L': 20})

In [181]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [182]:
models = [
    ClusterIsingFermion({'L': 20, 'n1': p}) for p in n1_pairs
]

In [183]:
type(models[0])

__main__.ClusterIsingFermion

In [184]:
models

In [24]:
mpo = models[0].calc_H_MPO()

In [30]:
mpo.get_W(0)

<npc.Array shape=(2, 3, 4, 4) labels=['wL', 'wR', 'p', 'p*']>

In [27]:
mpo.get_W(0).to_ndarray()

array([[[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]]])

In [29]:
mpo.get_W(1)

<npc.Array shape=(3, 3, 2, 2) labels=['wL', 'wR', 'p', 'p*']>

In [28]:
mpo.get_W(1).to_ndarray()

array([[[[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]]]])

In [31]:
mpo.get_W(2).to_ndarray()

array([[[[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  1.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  1.  ]],

        [[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]]],


       [[[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[-0.  , -0.  , -0.  , -0.  ],
         [-

In [15]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20)
    for m in models
]

In [16]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [17]:
for psi in psis:
    psi.canonical_form()

In [18]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [19]:
psis[0]

In [20]:
psis[0].get_B(0)

<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>

In [21]:
energies

[0.0, 0.0, 0.0, 0.0]

## Non-trivial proj rep

In [185]:
pauli_proj_rep_phases = np.array([
    [1, 1, 1, 1],
    [1, 1, -1, -1],
    [1, -1, 1, -1],
    [1, -1, -1, 1]
])

In [186]:
def get_proj_rep_term_phase(group_quad):
    g_left, g_in, g_out, g_right = group_quad

    g1 = mod_4_to_bit_addition(g_left, g_in)
    g2 = mod_4_to_bit_addition(g_in, g_right)
    g3 = mod_4_to_bit_addition(g_left, g_out)
    g4 = mod_4_to_bit_addition(g_out, g_right)

    numerator_phase = pauli_proj_rep_phases[g1, g2]
    denominator_phase = pauli_proj_rep_phases[g3, g4]

    return numerator_phase/denominator_phase

In [187]:
class ClusterFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [188]:
M = ClusterIsingFermion({'L': 20})

In [189]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [190]:
models = [
    [ClusterFermion({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [191]:
n1_pair_to_fermion_operators_dict

{(0, 0): 'Id', (0, 1): 'C', (1, 0): 'Cd', (1, 1): 'N'}

In [192]:
models

[[<__main__.ClusterFermion at 0x177bdb290>,

In [193]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [194]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [199]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
False
False
False
True
False
False
False


In [201]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 0
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [204]:
out

[[('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_2',

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [211]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 1
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [224]:
out

[[('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_2', 

In [226]:
[
    (l[1][0], l[2][0], l[3][0])
    for l in out
]

[('N', 'map_0_0', 'Id'),
 ('Cd', 'map_0_1', 'C'),
 ('N', 'map_0_2', 'Id'),
 ('Cd', 'map_0_3', 'C'),
 ('C', 'map_1_0', 'Cd'),
 ('Id', 'map_1_1', 'N'),
 ('C', 'map_1_2', 'Cd'),
 ('Id', 'map_1_3', 'N'),
 ('N', 'map_2_0', 'Id'),
 ('Cd', 'map_2_1', 'C'),
 ('N', 'map_2_2', 'Id'),
 ('Cd', 'map_2_3', 'C'),
 ('C', 'map_3_0', 'Cd'),
 ('Id', 'map_3_1', 'N'),
 ('C', 'map_3_2', 'Cd'),
 ('Id', 'map_3_3', 'N')]

In [228]:
left_fermion_operators = [l[1][0] for l in out]
right_fermion_operators = [l[3][0] for l in out]

In [229]:
[left_fermion_operators[4*i:4*i+4] for i in range(4)]

[['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id'],
 ['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id']]

In [230]:
[right_fermion_operators[4*i:4*i+4] for i in range(4)]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Check hermiticity

In [216]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            g_l = model_params.get('g_l', 0)
            g_r = model_params.get('g_r', 0)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_pairs = product(range(4), repeat=2)

            for group_pair in group_pairs:
                op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [217]:
models = [
    [
        [ClusterFermionHermiticityTest({'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r}) for p in n1_pairs]
        for g_l in range(4)
    ]
    for g_r in range(4)
]

In [220]:
mpos = [[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in models]

In [221]:
[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in mpos]

[[[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]]]

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Break down into further subcases

In [279]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2)

In [280]:
models = list()

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    m = ClusterFermionHermiticityTest(
                        {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                    )
                    
                    models[-1][-1][-1][-1].append(m)

In [281]:
models[0][0][0][0][0]

In [282]:
mpos = [[[[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [283]:
is_hermitian = np.array([[[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in mpos])

In [284]:
is_hermitian.shape

(4, 4, 4, 4, 4)

In [302]:
is_hermitian[1]

array([[[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]]],


       [[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ 

So breakdown in hermiticity is a function of $n_1$, $g_{in}$ and $g_{out}$, independent of $g_l$ and $g_r$.

So hermitian only if $g_{in}$ and $g_{out}$ are equal... i.e. it's the creation and annihilation terms causing problems.

In [310]:
n1_func = get_n1_func(0, 1)

g_l = 0
g_r = 0
g_in = 0
g_out = 2

In [321]:
get_op_list((g_l, 3, 3, g_r), n1_func)

[('map_0_0', 0, 0),
 ('N', 0, 1),
 ('map_3_3', 1, 0),
 ('N', 1, 1),
 ('map_0_0', 2, 0)]

In [322]:
op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)

In [323]:
op_list_1

[('map_0_0', 0, 0),
 ('C', 0, 1),
 ('map_0_2', 1, 0),
 ('C', 1, 1),
 ('map_0_0', 2, 0)]

In [324]:
op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)

In [325]:
op_list_2

[('map_0_0', 0, 0),
 ('Cd', 0, 1),
 ('map_2_0', 1, 0),
 ('Cd', 1, 1),
 ('map_0_0', 2, 0)]

Extract terms?

In [326]:
terms = models[1][0][0][2][0].all_coupling_terms()

In [327]:
terms

In [328]:
term_list = terms.to_TermList()

In [330]:
tl = term_list.terms

In [331]:
len(tl)

36

In [332]:
tl[0], tl[18]

([('map_0_0', 0), ('Cd JW', 1), ('map_2_0 JW', 2), ('Cd', 3), ('map_0_0', 4)],
 [('map_0_0', 0), ('C JW', 1), ('map_0_2 JW', 2), ('C', 3), ('map_0_0', 4)])

### Repeat but add hermitian conjugate terms

In [333]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1, plus_hc=True)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2, plus_hc=True)

In [339]:
models = list()
none_count = 0

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    try:
                        m = ClusterFermionHermiticityTest(
                            {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                        )
                    except ValueError:
                        m = None
                        none_count += 1
                    
                    models[-1][-1][-1][-1].append(m)

In [340]:
none_count

384

$384 = 128*3 \Rightarrow 1024/384 = 8/3$

Which ones are breaking...

In [341]:
models[0][0][0][0][0]

In [342]:
mpos = [[[[[None if m is None else m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [343]:
none_array = np.array([[[[[m is None for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models])

Looks like the cases I'm interested in are exactly those missing...!

In [349]:
none_array[1]

array([[[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]]],


       [[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [F

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [ ]:
models[0][1].c

In [195]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [196]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())
    
    for psi, m in zip(l1, l2):
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()

        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.181915
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.183525
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.188480
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.198468
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.188766
poorly conditioned H matrix in KrylovBased! |psi_0| = 0.187434
poorly conditioned H matrix in KrylovBased! |psi_0

In [197]:
energies

[[-18.000000000000036,
  -8.196875831824364,
  -8.514815527402932,
  -8.97384686708272],
 [-18.00000000000004,
  -8.628884646999314,
  -8.279348259530973,
  -8.999999999999979]]

## ZXZ case

In [105]:
def get_xzx_op_list(g_quad):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [136]:
class Cluster(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            sites = [spin,]
            return [spin,], ['s',]

        def init_terms(self, model_params):
            # Read off model parameters
            
            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_xzx_op_list(group_quad)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [137]:
models = [
    Cluster({'L': 20, 'non_trivial_proj_rep': b})
    for b in [True, False]
]

In [138]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [139]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0',],]*20) for m in models
]

In [140]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [141]:
for psi in psis:
    psi.canonical_form()

In [142]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [143]:
energies

[-17.999999999999996, -18.000000000000046]

In [144]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [152]:
m = models[0]

In [153]:
m.test_sanity()

In [154]:
mpo = m.calc_H_MPO()

In [155]:
mpo.is_hermitian()

True

In [156]:
m = models[1]

In [157]:
m.test_sanity()

In [158]:
mpo = m.calc_H_MPO()

In [159]:
mpo.is_hermitian()

True

In [160]:
psi = outpsis[1]

In [163]:
psi.get_B(0).to_ndarray()

array([[[1.],
        [0.],
        [0.],
        [0.]]])

In [164]:
psi.get_B(1).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])

In [165]:
psi.get_B(10).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])